<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/cuaderno_de_entrenamiento_VITS_en_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuaderno para entrenamiento de [VITS](https://github.com/jaywalnut310/vits)

Cuaderno y soporte para el Español desarrollados por [rmcpantoja](https://github.com/rmcpantoja/), en colaboración con [Mixomo](https://github.com/Mixomo/efficient-vits-finetuning-Spanish-support-WIP-)

## créditos:

VITS: Conditional Variational Autoencoder with Adversarial Learning for End-to-End Text-to-Speech. (Jaehyeon Kim, Jungil Kong, y Juhee Son, 2021): https://arxiv.org/abs/2106.06103

## Primeros pasos

In [ ]:
#@title Comprobar la GPU asignada

#@markdown Una GPU con mayor capacidad puede llebar mayor velocidad de respuesta.
!nvidia-smi -L

In [ ]:
#@title Montar Google Drive
#@markdown Necesitas como mínimo 1-2 GB de espacio libre.
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@title Instalar software

#@markdown En esta celda se instalará el sintetizador y sus componentes necesarios para ejecutar el entrenamiento. (esto puede tomar un tiempo)

#@markdown **Nota: por ahora, el fine-tuning/ajuste de tus datasets se entrenarán usando el modelo Inglés de LJSpeech como base. Estamos en camino para modelos preentrenados en español, por lo que podrá mejorar la capacidad de entrenamiento. Entonces, necesitarás mucha cantidad de datos para que funcione decente.**
%cd /content
!git clone https://github.com/Mixomo/efficient-vits-finetuning-Spanish-support-WIP- vits
%cd vits
!cp /usr/local/cuda/lib64/libcudart.so.11.0 /usr/lib64-nvidia/libcudart.so.11.0
# FOR ORIGINAL REPO:
#!pip install Cython librosa==0.8.0 matplotlib==3.3.1 numpy phonemizer scipy==1.7.2 tensorboard Unidecode torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -U
#FOR FINE-TUNE REPO:
!pip install Cython librosa==0.9.1 matplotlib numpy scipy tensorboard unidecode protobuf tqdm phonemizer bitsandbytes wandb num2words
%cd monotonic_align
!python setup.py build_ext --inplace
!sudo apt-get install espeak-ng
%cd ..
# descargar procesador JSON:
!wget https://github.com/mikefarah/yq/releases/download/v4.29.2/yq_linux_amd64.tar.gz
!tar -xvf yq_linux_amd64.tar.gz
#actualizar gdown y descargar modelos preentrenados (ljspeech inglés, español pronto):
!pip install --upgrade gdown
import gdown
gdown.download("https://drive.google.com/file/d/1T-u3OV49W6Lv3bDxh-EA63ALZKHqyy0t/view?usp=sharing", "/content/vits/pretrained/generator.pth", quiet=False, fuzzy=True)
gdown.download("https://drive.google.com/file/d/118ffn807Eqlu891qbNRQP7O9E0-aMPxM/view?usp=sharing", "/content/vits/pretrained/discriminator.pth", quiet=False, fuzzy=True)
# check bitsandbytes
!python -m bitsandbytes
# for dataset:
!mkdir /content/vits/wavs

## entrenamiento

In [ ]:
#@title 1. Extrae el conjunto de datos
#@markdown Importante: los audios deberán estar en formato wav, (22050hz, 16-bits, mono), y, para comodidad, enumerados. Ejemplo:

#@markdown * 1.wav
#@markdown * 2.wav
#@markdown * 3.wav
#@markdown * .....

#@markdown ---

#@markdown #### Ruta de audios. (estos deberán estar en un archivo comprimido zip, sueltos)
wavs_path = "/content/drive/MyDrive/Fakeyou/aldEnhanced/wavs.zip" #@param {type:"string"}
!unzip -q "{wavs_path}" -d /content/vits/wavs

In [ ]:
#@title 2. Subir y procesar la transcripción
#@markdown importante: la transcripción significa escribir lo que dice el personaje en cada uno de los audios, y deberá tener la siguiente estructura:

#@markdown * wavs/1.wav|Cuando los españoles llegaron a América se encontraron con montañas imponentes, ríos inmensos, aborígenes desconocidos, feroces algunos, muy apacibles otros.
#@markdown * wavs/2.wav|Al influjo de los elementos de naturaleza tan grandiosa y de sus extraordinarios habitantes, los conquistadores empezaron a tejer una serie de mitos y leyendas para justificar su existencia.
#@markdown * wavs/3.wav|La de las amazonas es una de ellas, y cuenta que hace mucho tiempo, la tribu de los Worisiana se instaló a orillas del poderoso río-mar.
#@markdown * ...............

#@markdown Y así sucesibamente. Además, la transcripción deberá tener un formato .txt (UTF8 sin bom)

from google.colab import files
import random
%cd /content/vits/filelists
!rm /content/vits/filelists/list.txt
listfn, length = files.upload().popitem()
if listfn != "list.txt":
  !mv "$listfn" list.txt
%cd ..
print("¡Transcripción subida! dibidiendo la lista en entrenamiento y validación...")
with open("filelists/list.txt", encoding="utf-8") as f:
  original = f.read().split('\n')

ratio = 0.01
def split_data(data, ratio):
  train_index = list(range(len(data)))
  val_index = []
  while len(val_index) < ratio * len(train_index):
    val_index.append(train_index.pop(random.randint(0, len(train_index)-1)))
  return [data[i] for i in train_index], [data[i] for i in val_index]

train, val = split_data(original, ratio)

with open("filelists/list.txt", "w", encoding="utf-8") as f:
  f.write('\n'.join(train))

with open("filelists/list_val.txt", "w", encoding="utf-8") as f:
  f.write('\n'.join(val))
print("¡Completado! Procesando el texto a fonemas...")
!python preprocess.py --filelists "filelists/list.txt" "filelists/list_val.txt" --text_cleaners "spanish_cleaners"
print("¡Todo listo!")

In [ ]:
#@title 3. Configuración
config_path = "configs/ljs_base.json"
#@markdown #### Nombre deseado para el modelo:
model_name = "Aldo" #@param {type:"string"}
!./yq_linux_amd64 -i '.data.name = "{model_name}"' "{config_path}"
#@markdown ---
#@markdown #### carpeta de salida (no se recomienda cambiar):
output_path = "/content/drive/MyDrive/colab/vits" #@param {type:"string"}
#@markdown ---
#@markdown #### Frecuencia de muestreo (opcional):
sample_rate = "22050" #@param ["22050", "32000", "44100"]
!./yq_linux_amd64 -i '.data.sampling_rate = {sample_rate}' "{config_path}"
#@markdown ---
#@markdown #### épocas de entrenamiento:
train_epochs = 200 #@param {type:"integer"}
!./yq_linux_amd64 -i '.train.epochs = {train_epochs}' "{config_path}"
#@markdown ---
##@markdown #### tasa de aprendizaje (no se recomienda cambiar):
#learning_rate = 2e-4
#!./yq_linux_amd64 -i '.train.learning_rate = {learning_rate}' "{config_path}"
##@markdown ---
#@markdown #### Tamaño del lote:
batch_size = 12 #@param {type:"integer"}
!./yq_linux_amd64 -i '.train.batch_size = {batch_size}' "{config_path}"
#@markdown ---
#@markdown #### Resumir wandb?
wandb_resume = False #@param {type:"boolean"}
if wandb_resume:
    resume = "true"
else:
    resume = "false"
!./yq_linux_amd64 -i '.train.wandb_resume = {resume}' "{config_path}"
#@markdown ---
# train and val:
!./yq_linux_amd64 -i '.data.training_files = "filelists/list.txt.cleaned"' "{config_path}"
!./yq_linux_amd64 -i '.data.validation_files = "filelists/list_val.txt.cleaned"' "{config_path}"
# cleaner:
!./yq_linux_amd64 -i '.data.text_cleaners = ["spanish_cleaners"]' "{config_path}"

In [ ]:
#@title 4. Ejecutar la extensión TensorBoard
import os
if not os.path.exists("/content/drive/MyDrive/colab/vits"):
    os.makedirs("/content/drive/MyDrive/colab/vits")
%load_ext tensorboard
%tensorboard --logdir "{output_path}"

In [ ]:
#@title 5. Comenzar el entrenamiento
#@markdown Importante: si entrenas desde cero, recuerda eliminar los respaldos anteriores de modelos (si se encuentran) para evitar mayor consumo de espacio.

!python train.py --config "{config_path}" --model "{model_name}" --path "{output_path}"

# ¿Curioso por escuchar cómo suena el modelo?

¡Prueba el cuaderno de síntesis [aquí](https://colab.research.google.com/drive/1mlqngf8t6NUMAD35A8dimMEe2_sXdYAL)!

# Cuadernos relacionados/de interés

* [Cuaderno completo de SoftVC VITS Singing Voice Conversion (entrenamiento+inferencia)](https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/Cuaderno_completo_So_Vits_SVC_en_espa%C3%B1ol.ipynb)